In [1]:
"""
Santa 2025 - EXACT COMPETITION TREE SHAPE
Single-cell Kaggle solution with official tree geometry
Copy this entire file into a Kaggle notebook cell and run!
"""

import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely import affinity
from dataclasses import dataclass
from typing import List, Tuple
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# ============ OFFICIAL COMPETITION TREE SHAPE ============
def create_competition_tree_shape():
    """
    Official Christmas tree shape from competition starter notebook
    Multi-tiered tree with trunk at bottom
    """
    scale_factor = 1.0
    
    # Tree dimensions (from competition)
    trunk_w = 0.15
    trunk_h = 0.2
    base_w = 0.7
    mid_w = 0.4
    top_w = 0.25
    tip_y = 0.8
    tier_1_y = 0.5
    tier_2_y = 0.25
    base_y = 0.0
    trunk_bottom_y = -trunk_h
    
    # Exact vertices from competition (scaled)
    vertices = np.array([
        # Tip
        [0.0 * scale_factor, tip_y * scale_factor],
        # Right side - Top Tier
        [top_w / 2 * scale_factor, tier_1_y * scale_factor],
        [top_w / 4 * scale_factor, tier_1_y * scale_factor],
        # Right side - Middle Tier
        [mid_w / 2 * scale_factor, tier_2_y * scale_factor],
        [mid_w / 4 * scale_factor, tier_2_y * scale_factor],
        # Right side - Bottom Tier
        [base_w / 2 * scale_factor, base_y * scale_factor],
        # Right Trunk
        [trunk_w / 2 * scale_factor, base_y * scale_factor],
        [trunk_w / 2 * scale_factor, trunk_bottom_y * scale_factor],
        # Left Trunk
        [-trunk_w / 2 * scale_factor, trunk_bottom_y * scale_factor],
        [-trunk_w / 2 * scale_factor, base_y * scale_factor],
        # Left side - Bottom Tier
        [-base_w / 2 * scale_factor, base_y * scale_factor],
        # Left side - Middle Tier
        [-mid_w / 4 * scale_factor, tier_2_y * scale_factor],
        [-mid_w / 2 * scale_factor, tier_2_y * scale_factor],
        # Left side - Top Tier
        [-top_w / 4 * scale_factor, tier_1_y * scale_factor],
        [-top_w / 2 * scale_factor, tier_1_y * scale_factor],
    ])
    
    return vertices

print("Competition tree shape loaded!")
print("Tree dimensions: 0.7 wide × 1.0 tall (tip to trunk bottom)")

# ============ CORE CLASSES ============
@dataclass
class TreeShape:
    vertices: np.ndarray
    def to_polygon(self): return Polygon(self.vertices)
    def get_rotated(self, deg): return affinity.rotate(self.to_polygon(), deg, origin=(0,0))
    def get_transformed(self, x, y, deg): return affinity.translate(self.get_rotated(deg), xoff=x, yoff=y)

@dataclass
class PlacedTree:
    tree_id: int
    x: float
    y: float
    deg: float
    polygon: Polygon

class TreePacker:
    def __init__(self, tree_shape):
        self.tree_shape = tree_shape
        self.placed_trees = []
    
    def check_collision(self, poly):
        return any(poly.intersects(p.polygon) and poly.intersection(p.polygon).area > 1e-6 
                   for p in self.placed_trees)
    
    def place_tree(self, tree_id, x, y, deg):
        poly = self.tree_shape.get_transformed(x, y, deg)
        if self.check_collision(poly): return False
        self.placed_trees.append(PlacedTree(tree_id, x, y, deg, poly))
        return True
    
    def get_box_size(self):
        coords = np.vstack([np.array(p.polygon.exterior.coords) for p in self.placed_trees])
        return max(coords.max(axis=0) - coords.min(axis=0))
    
    def center(self):
        coords = np.vstack([np.array(p.polygon.exterior.coords) for p in self.placed_trees])
        center = (coords.max(axis=0) + coords.min(axis=0)) / 2
        for p in self.placed_trees:
            p.x -= center[0]
            p.y -= center[1]
            p.polygon = self.tree_shape.get_transformed(p.x, p.y, p.deg)
    
    def clear(self):
        self.placed_trees = []

# ============ GRID PACKING ============
def grid_pack(tree_shape, n, angles=[0,30,45,60,90]):
    best_score, best = float('inf'), []
    packer = TreePacker(tree_shape)
    
    for cols in range(1, n+1):
        rows = (n + cols - 1) // cols
        for angle in angles:
            packer.clear()
            bounds = tree_shape.get_rotated(angle).bounds
            sx, sy = (bounds[2]-bounds[0])*1.1, (bounds[3]-bounds[1])*1.1
            
            tid = 0
            for r in range(rows):
                for c in range(cols):
                    if tid >= n: break
                    if not any(packer.place_tree(tid, c*sx, r*sy, angle+off) for off in [0,5,-5]):
                        tid = -1
                        break
                    tid += 1
                if tid < 0: break
            
            if len(packer.placed_trees) == n:
                packer.center()
                score = (packer.get_box_size()**2) / n
                if score < best_score:
                    best_score, best = score, [PlacedTree(p.tree_id,p.x,p.y,p.deg,p.polygon) for p in packer.placed_trees]
    
    return best_score, best

# ============ GENETIC ALGORITHM ============
def genetic_pack(tree_shape, n, pop_size=50, gens=100):
    packer = TreePacker(tree_shape)
    
    def evaluate(genes):
        packer.clear()
        for i in range(n):
            if not packer.place_tree(i, genes[i*3], genes[i*3+1], genes[i*3+2]%360):
                return 1e6, []
        packer.center()
        return (packer.get_box_size()**2)/n, [PlacedTree(p.tree_id,p.x,p.y,p.deg,p.polygon) for p in packer.placed_trees]
    
    pop = [np.concatenate([np.random.randn(n*2)*2, np.random.uniform(0,360,n)]).reshape(-1,3).flatten() for _ in range(pop_size)]
    fits = [evaluate(ind) for ind in pop]
    best_idx = min(range(len(fits)), key=lambda i: fits[i][0])
    best_score, best = fits[best_idx]
    
    for gen in range(gens):
        selected = [pop[min(np.random.choice(len(pop), 5), key=lambda i: fits[i][0])].copy() for _ in range(pop_size)]
        next_pop = []
        for i in range(0, len(selected), 2):
            if i+1 < len(selected):
                pt = np.random.randint(1, len(selected[i]))
                c1, c2 = np.concatenate([selected[i][:pt], selected[i+1][pt:]]), np.concatenate([selected[i+1][:pt], selected[i][pt:]])
                for c in [c1, c2]:
                    for j in range(len(c)):
                        if np.random.random() < 0.1:
                            c[j] = np.random.uniform(0,360) if j%3==2 else c[j]+np.random.normal(0,0.5)
                next_pop.extend([c1, c2])
        
        for idx in sorted(range(len(fits)), key=lambda i: fits[i][0])[:5]:
            next_pop.append(pop[idx].copy())
        
        pop = next_pop[:pop_size]
        fits = [evaluate(ind) for ind in pop]
        idx = min(range(len(fits)), key=lambda i: fits[i][0])
        if fits[idx][0] < best_score:
            best_score, best = fits[idx]
        
        if gen % 20 == 0:
            print(f"    Gen {gen}: best={best_score:.4f}")
    
    return best_score, best

# ============ VISUALIZATION ============
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def visualize_solution(placement, n, score, strategy, save=True):
    """Visualize a single configuration"""
    fig, ax = plt.subplots(figsize=(8, 8))
    
    if not placement:
        ax.text(0.5, 0.5, 'No solution', ha='center', va='center', fontsize=16)
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        return fig
    
    # Colors for trees
    colors = plt.cm.tab20(np.linspace(0, 1, len(placement)))
    
    # Draw each tree
    for i, tree in enumerate(placement):
        coords = np.array(tree.polygon.exterior.coords)
        poly = mpatches.Polygon(coords, closed=True,
                               facecolor=colors[i], 
                               edgecolor='darkgreen',
                               linewidth=1.5, alpha=0.7)
        ax.add_patch(poly)
        
        # Label
        centroid = tree.polygon.centroid
        ax.text(centroid.x, centroid.y, str(tree.tree_id),
               ha='center', va='center', fontsize=8, fontweight='bold')
    
    # Get bounding box
    all_coords = []
    for tree in placement:
        coords = np.array(tree.polygon.exterior.coords)
        all_coords.extend(coords)
    all_coords = np.array(all_coords)
    
    min_x, min_y = all_coords.min(axis=0)
    max_x, max_y = all_coords.max(axis=0)
    box_size = max(max_x - min_x, max_y - min_y)
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    
    # Draw bounding box
    rect = mpatches.Rectangle(
        (center_x - box_size/2, center_y - box_size/2),
        box_size, box_size,
        linewidth=2, edgecolor='red', facecolor='none', linestyle='--'
    )
    ax.add_patch(rect)
    
    # Set limits
    padding = box_size * 0.1
    ax.set_xlim(center_x - box_size/2 - padding, center_x + box_size/2 + padding)
    ax.set_ylim(center_y - box_size/2 - padding, center_y + box_size/2 + padding)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_title(f"{n} Trees | Score: {score:.3f} | Strategy: {strategy}\nBox: {box_size:.3f}", 
                fontsize=12, fontweight='bold')
    
    if save:
        plt.savefig(f'config_{n:03d}_trees.png', dpi=150, bbox_inches='tight')
        plt.close(fig)
    
    return fig

def visualize_comparison(solutions, configs_to_show=None):
    """Create comparison grid of multiple configurations"""
    if configs_to_show is None:
        configs_to_show = [1, 2, 3, 5, 10, 20, 50, 100]
    
    # Filter to available configs
    configs_to_show = [n for n in configs_to_show if n in solutions]
    if not configs_to_show:
        return
    
    n_configs = len(configs_to_show)
    cols = 4
    rows = (n_configs + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(20, 5 * rows))
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for idx, n in enumerate(configs_to_show):
        row = idx // cols
        col = idx % cols
        ax = axes[row, col]
        
        score, placement, strategy = solutions[n]
        
        if not placement:
            ax.text(0.5, 0.5, f'{n} trees\n(no solution)', 
                   ha='center', va='center')
            ax.set_xlim(0, 1)
            ax.set_ylim(0, 1)
            ax.axis('off')
            continue
        
        # Draw trees
        colors = plt.cm.tab20(np.linspace(0, 1, len(placement)))
        for i, tree in enumerate(placement):
            coords = np.array(tree.polygon.exterior.coords)
            poly = mpatches.Polygon(coords, closed=True,
                                   facecolor=colors[i],
                                   edgecolor='black',
                                   linewidth=1, alpha=0.7)
            ax.add_patch(poly)
        
        # Bounding box
        all_coords = []
        for tree in placement:
            coords = np.array(tree.polygon.exterior.coords)
            all_coords.extend(coords)
        all_coords = np.array(all_coords)
        
        min_x, min_y = all_coords.min(axis=0)
        max_x, max_y = all_coords.max(axis=0)
        box_size = max(max_x - min_x, max_y - min_y)
        center_x = (min_x + max_x) / 2
        center_y = (min_y + max_y) / 2
        
        rect = mpatches.Rectangle(
            (center_x - box_size/2, center_y - box_size/2),
            box_size, box_size,
            linewidth=2, edgecolor='red', facecolor='none', linestyle='--'
        )
        ax.add_patch(rect)
        
        padding = box_size * 0.15
        ax.set_xlim(center_x - box_size/2 - padding, center_x + box_size/2 + padding)
        ax.set_ylim(center_y - box_size/2 - padding, center_y + box_size/2 + padding)
        ax.set_aspect('equal')
        ax.grid(True, alpha=0.3)
        ax.set_title(f"{n} trees\nScore: {score:.3f} ({strategy})", fontsize=10)
    
    # Hide unused subplots
    for idx in range(n_configs, rows * cols):
        row = idx // cols
        col = idx % cols
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.savefig('comparison_grid.png', dpi=150, bbox_inches='tight')
    plt.close(fig)
    print("✓ Saved: comparison_grid.png")

# ============ SOLVER ============
def solve(tree_shape, n):
    print(f"n={n:3d}:", end=" ")
    
    grid_score, grid_place = grid_pack(tree_shape, n)
    results = [('grid', grid_score, grid_place)]
    print(f"grid={grid_score:.4f}", end=" ")
    
    if n <= 30:  # Use GA for small configs
        ga_score, ga_place = genetic_pack(tree_shape, n, pop_size=max(30,n*2), gens=min(100,200//n))
        if ga_place:
            results.append(('ga', ga_score, ga_place))
            print(f"ga={ga_score:.4f}", end=" ")
    
    results.sort(key=lambda x: x[1])
    strategy, score, placement = results[0]
    print(f"→ {strategy} wins")
    return strategy, score, placement

# ============ CREATE TREE ============
tree_vertices = create_competition_tree_shape()
tree = TreeShape(tree_vertices)

# ============ TEST ON SMALL RANGE ============
print("\n" + "="*60)
print("TESTING ON 1-10 TREES (OFFICIAL TREE SHAPE)")
print("="*60)

test_solutions = {}
for n in range(1, 11):
    strategy, score, placement = solve(tree, n)
    test_solutions[n] = (score, placement, strategy)

test_score = sum(s[0] for s in test_solutions.values())
print("="*60)
print(f"Test score (1-10): {test_score:.4f}")
print("="*60)

# Count strategy wins
from collections import Counter
strategies = [s[2] for s in test_solutions.values()]
strategy_counts = Counter(strategies)
print(f"\nStrategy wins: {dict(strategy_counts)}")

# ============ VISUALIZATIONS ============
print("\n" + "="*60)
print("GENERATING VISUALIZATIONS")
print("="*60)

# Individual configs
for n in [1, 3, 5, 10]:
    score, placement, strategy = test_solutions[n]
    visualize_solution(placement, n, score, strategy)
    print(f"✓ Saved: config_{n:03d}_trees.png")

# Comparison grid
visualize_comparison(test_solutions, configs_to_show=list(range(1, 11)))

# ============ GENERATE TEST SUBMISSION ============
rows = []
for n, (score, placement, _) in test_solutions.items():
    for tree in placement:
        rows.append({
            'id': f"{n:03d}_{tree.tree_id}",
            'x': f"s{tree.x:.10f}",
            'y': f"s{tree.y:.10f}",
            'deg': f"s{tree.deg:.10f}"
        })

df = pd.DataFrame(rows)
df.to_csv('submission_test.csv', index=False)
print(f"\n✓ Test submission saved: {len(rows)} trees")
print("\nFirst 10 rows:")
print(df.head(10))

# ============ UNCOMMENT TO RUN FULL (1-200) ============
# print("\n" + "="*60)
# print("RUNNING FULL COMPETITION (1-200 TREES)")
# print("="*60)
# print("This will take 30-60 minutes...\n")
# 
# full_solutions = {}
# for n in range(1, 201):
#     strategy, score, placement = solve(tree, n)
#     full_solutions[n] = (score, placement, strategy)
#     
#     # Visualize key configs
#     if n in [1, 2, 3, 5, 10, 20, 50, 100, 150, 200]:
#         visualize_solution(placement, n, score, strategy)
#         print(f"    ✓ Saved visualization for n={n}")
#     
#     if n % 10 == 0:
#         partial_score = sum(full_solutions[i][0] for i in range(1, n+1))
#         print(f"Progress: {n}/200 | Cumulative score: {partial_score:.4f}")
# 
# total_score = sum(s[0] for s in full_solutions.values())
# print("="*60)
# print(f"FINAL SCORE: {total_score:.4f}")
# print("="*60)
# 
# # Generate comparison grids
# print("\nGenerating comparison visualizations...")
# visualize_comparison(full_solutions, configs_to_show=[1,2,3,5,10,20,30,40])
# visualize_comparison(full_solutions, configs_to_show=[50,60,80,100,120,150,180,200])
# 
# # Generate final submission
# rows = []
# for n, (score, placement, _) in full_solutions.items():
#     for tree in placement:
#         rows.append({
#             'id': f"{n:03d}_{tree.tree_id}",
#             'x': f"s{tree.x:.10f}",
#             'y': f"s{tree.y:.10f}",
#             'deg': f"s{tree.deg:.10f}"
#         })
# 
# df = pd.DataFrame(rows)
# df.to_csv('submission.csv', index=False)
# print(f"\n✓ FINAL SUBMISSION: {len(rows)} trees")
# print("Download 'submission.csv' and submit to Kaggle!")
# print("\n✓ Generated visualizations:")
# print("  - config_XXX_trees.png (for key configs)")
# print("  - comparison_grid.png (multiple comparison grids)")

print("\n" + "="*60)
print("READY TO GO!")
print("="*60)
print("Next: Uncomment the 'RUN FULL' section above and run again")

Competition tree shape loaded!
Tree dimensions: 0.7 wide × 1.0 tall (tip to trunk bottom)

TESTING ON 1-10 TREES (OFFICIAL TREE SHAPE)
n=  1: grid=0.6613     Gen 0: best=0.6623
    Gen 20: best=0.6623
    Gen 40: best=0.6623
    Gen 60: best=0.6619
    Gen 80: best=0.6619
ga=0.6619 → grid wins
n=  2: grid=1.0804     Gen 0: best=79.9796
    Gen 20: best=4.7036
    Gen 40: best=0.6049
    Gen 60: best=0.5212
    Gen 80: best=0.5212
ga=0.5212 → ga wins
n=  3: grid=0.9720     Gen 0: best=672.6196
    Gen 20: best=196.1322
    Gen 40: best=117.0167
    Gen 60: best=70.3233
ga=58.1721 → grid wins
n=  4: grid=0.7290     Gen 0: best=3406.3988
    Gen 20: best=2948.4627
    Gen 40: best=2622.3161
ga=2467.4958 → grid wins
n=  5: grid=1.0035     Gen 0: best=1951.8555
    Gen 20: best=1654.2594
ga=1425.6273 → grid wins
n=  6: grid=0.8363     Gen 0: best=6225.2132
    Gen 20: best=5859.1773
ga=5643.7283 → grid wins
n=  7: grid=0.9673     Gen 0: best=440.0919
    Gen 20: best=323.0398
ga=307.6642 → 